In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

from dataclasses import dataclass
INF = 1e9 + 7777
@dataclass
class Node :
  i: int = -1
  v: int = INF

class ST : 
  def __init__(self, L):
    self._size = _size = 1 << (len(L) - 1).bit_length()
    self.seg = [Node() for _ in range(2 * _size)]
    self.seg[_size:_size + len(L)] = [Node(i = i, v = v) for i, v in enumerate(L)]
    self.init()

  def merge(self, l: Node, r: Node):
    ret = Node()
    if r.i == -1 :
      ret.i = l.i
      ret.v = l.v
    elif l.i == -1 :
      ret.i = r.i
      ret.v = r.v
    elif r.v < l.v :
      ret.i = r.i
      ret.v = r.v
    elif r.v == l.v : #같은 경우 = 인덱스가 작은 쪽의 인덱스로 설정
      ret.i = min(l.i, r.i)
      ret.v = l.v
    else :
      ret.i = l.i
      ret.v = l.v
    return ret

  def init(self):
    for i in reversed(range(self._size)) :
      L = self.seg[i*2]
      R = self.seg[i*2+1]
      self.seg[i] = self.merge(L, R)

  def __getitem__(self, i):
    return self.seg[i + self._size]

  def update(self, i, v) :
    i += self._size
    self.seg[i].v = v
    while i > 1 :
      i >>= 1
      self.seg[i] = self.merge(self.seg[i*2], self.seg[i*2+1])

  def query(self, s, e): #[s, e)
    s += self._size
    e += self._size

    l = Node()
    r = Node()
    while s < e :
      if s & 1 :
        l = self.merge(l, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        r = self.merge(self.seg[e], r)
      s >>= 1
      e >>= 1
    return self.merge(l, r)

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  st = ST(L)
  M = int(input())
  ans = []
  for _ in range(M) :
    q, i, j = map(int, input().split())
    match q:
      case 1: st.update(i-1, j)
      case 2: ans.append(st.query(i-1, j).i + 1)

  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### 아이디어 
- 구조체를 활용한 세그먼트 트리라는 것을 10분정도만에 발상했다. 
  - 노드에, 값의 최솟값과 index의 최솟값을 저장하게 하고, 쿼리마다 최솟값을 반환하게 하면 된다.
- 오히려 더 어려운 문제를 먼저 풀었으므로 구현은 쉬울 것 같았다.

### 풀이
- 아이디어는 완벽히 맞았는데, 거지같이 만든 템플릿 때문에 이상한곳에서 엄청나게 디버깅 했다. 
  - 그냥 구간 관련은 [s, e)로 고정하는 것이 무조건 옳다..